##             Prediction de la valeur fonciere  à l'aide de model XGBRegressor

### 1- Importation des librairies et du fichier de données en csv

In [ ]:
# !pip install xgboost 

In [ ]:
#Importation des librairies et du fichier de données en csv:

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
pd.pandas.set_option('display.max_columns',None)


from sklearn import model_selection, preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor


from scipy import stats


import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importation de base de données
path = '/content/drive/MyDrive/01-Abderaouf/IDF_2019_to_2021_v3.csv'
df = pd.read_csv(path)
# df = pd.read_csv('IDF_2019_to_2021_v3.csv')

### 3-a Remplacement des valeur manquante longitude latitude 
- Remplissage vers l'avant

- Remplissage arrière


In [ ]:
df['longitude'] = df.groupby('code_commune')['longitude'].bfill().ffill()
df['latitude'] = df.groupby('code_commune')['latitude'].bfill().ffill()

df.latitude.isna().sum()

0

### 3-b  Choix des variables pertinents selon l'expertise métier 

 Nous allons supprimer les variables identifiées comme non pertinentes, et assurer que le code départeme est du type catégorielle avant d'entamer le prétraitement des données 

In [ ]:
df2 =df.drop(['id_mutation','date_mutation',
               'numero_disposition','nature_mutation','adresse_numero',
               'adresse_suffixe','adresse_nom_voie','adresse_code_voie','longitude','latitude','section_cadastrale',
              'nom_commune','annee','code_annee','Appartement','Dépendance','Maison','id_parcelle','code_postal',
              'prix_m²','prix_m²_moy_co','prix_m²_moy_ca'], axis = 1)

df2['code_departement'] = df2['code_departement'].astype('category')

3-c Niveau de vie 

In [ ]:
path_2 = '/content/drive/MyDrive/01-Abderaouf/FRANCE_COMMUNE_NIVEAU_DE_VIE-FIGARO.csv'

niv_vie = pd.read_csv(path_2, sep= ';')

niv_vie.rename({'CODGEO':'code_commune'},axis=1,inplace=True)

niv_vie['MED14'] = niv_vie['MED14'].str.replace('[A-Za-z]', '').str.replace(',', '.').astype(float)

niv_vie['MED14'] = niv_vie.groupby('code_commune')['MED14'].bfill().ffill()


niv_vie.rename({'CODGEO':'code_commune'},axis=1,inplace=True)


niv_vie['code_commune'] = niv_vie['code_commune'].apply(lambda x: x.replace("A", "0"))
niv_vie['code_commune'] = niv_vie['code_commune'].apply(lambda x: x.replace("B", "0"))

niv_vie['code_commune'] = niv_vie['code_commune'].astype('float')
df2['code_commune'] = df2['code_commune'].astype('float')

df2 = pd.merge(df2,niv_vie, on = 'code_commune', how ='left')

df2.sample()

,valeur_fonciere,code_commune,code_departement,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,terrains_a_batir,terrains_autre,type_bien,LIBGEO,MED14
269143,169000.0,75109.0,75,2,16.0,1.0,0.0,0,0,Appartement,Paris 9e Arrondissement,32319.0


In [ ]:
!pip install --upgrade geopandas libpysal contextily cenpy osmnx rasterio 
import geopandas as gpd
path_2 = '/content/drive/MyDrive/01-Abderaouf/centroide_communes.json'
geo_commune = gpd.read_file(path_2)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 2.4 MB 58.9 MB/s 
     |████████████████████████████████| 95 kB 5.1 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 6.3 MB 44.0 MB/s 
     |████████████████████████████████| 16.7 MB 48.3 MB/s 
     |████████████████████████████████| 1.0 MB 22.7 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 11.2 MB 62.6 MB/s 
     |████████████████████████████████| 944 kB 47.4 MB/s 
  Created wheel for cenpy: filename=cenpy-1.0.0.post4-py3-none-any.whl size=29924 sha256=b4e2bfba49f3a0eb3bddc002ef6a3594387dcbd8bbda1a16e1af2a03da1943d5
  Stored in directory: /root/.cache/pip/wheels/3c/6a/eb/8435134ce3552e7d9907378efdc19f5c167726dea64b8185d9
Successfully built cenpy
  Attempting uninstall: requests
    F

In [ ]:
geo_commune.drop(['nom_com','nom_com_m','statut','insee_arr','insee_reg','code_epci'], axis=1, inplace=True)
geo_commune.rename({'insee_com':'code_commune'},axis=1,inplace=True)

geo_commune['code_commune'] = geo_commune['code_commune'].apply(lambda x: x.replace("A", "0"))
geo_commune['code_commune'] = geo_commune['code_commune'].apply(lambda x: x.replace("B", "0"))

geo_commune["x"] = geo_commune.centroid.map(lambda p: p.x)
geo_commune["y"] = geo_commune.centroid.map(lambda p: p.y)


geo_commune['code_commune'] = geo_commune['code_commune'].astype('float')

df2 = pd.merge(df2,geo_commune, on = 'code_commune', how ='left')
df2.drop(['LIBGEO','code_commune','id','insee_dep','population','X_COORD','Y_COORD','geometry'], axis=1,inplace=True)

In [ ]:
df2.isna().sum()
# df2= df2[(df2['valeur_fonciere']<1000000) & (df2['valeur_fonciere']>30000)]
# df2.drop(['LIBGEO','code_commune'], axis=1,inplace=True)

valeur_fonciere              0
code_departement             0
nombre_lots                  0
surface_reelle_bati          0
nombre_pieces_principales    0
surface_terrain              0
terrains_a_batir             0
terrains_autre               0
type_bien                    0
MED14                        0
x                            0
y                            0
dtype: int64

In [ ]:
df2.sample()

,valeur_fonciere,code_departement,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,terrains_a_batir,terrains_autre,type_bien,MED14,x,y
207205,162000.0,92,2,21.0,1.0,0.0,0,0,Appartement,20851.3,2.247051,48.922752


3-d Centroid commune

In [ ]:
# path_export = '/content/drive/MyDrive/01-Abderaouf/IDF_2019_to_2021_v4.csv'
# df2.to_csv(path_export)


## 4-Prétraitement des donneés  
### 4-a OneHotEncoder (column_transformer)

Nous allons au premier temps  encoder les colonnes code departement et type bien en valeurs numérique

Nous avons fait le choix d'encoder que ces deux colonnes catégorielles pour une première Modélisation

le nombre de modalités de code communes est très élevé (plus de 1200), l'encodage de cette variable rajoute un grand nombre d'itération pour le modèle qui impacte drastiquement le temps de traitement et de mémoire.

In [ ]:
ohe = OneHotEncoder()

column_trans = make_column_transformer(
    (OneHotEncoder(), ['code_departement', 'type_bien']),
    remainder='passthrough')

### 4-b PolynomialFeatures | StandarScaler  (FeatureUnion)

Nous allons appliquer un traitement PolynomialFeatures d'ordre 2 sur nos données, ensuite nous allons standardiser les variables avant la création de notre modèle Elastic Net

In [ ]:
poly = PolynomialFeatures(degree=2,include_bias = False) # Calcule des features polynomiaux d'ordre 2
std_scaler = StandardScaler()    # Centrage et réduction

# union = FeatureUnion([ ('Standarisaion', std_scaler),('poly_features', poly)])
union = FeatureUnion([ ('Standarisaion', std_scaler)])

###  5- Création de Model  XGBRegressor avec l'initiation des valuer alpha et l1_ratio

In [ ]:
# elastic_net = ElasticNetCV(l1_ratio =(0.1, 0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99),
#                         alphas = (0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 50, 100), cv=10)
# RFR = RandomForestRegressor(n_estimators=400, max_depth=100, random_state=0)
# lasso = LassoCV(alphas = (0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 50, 100),cv=10)
# ridge = RidgeCV(alphas= (0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 50, 100), cv=10)
# GBR=GradientBoostingRegressor(n_estimators=450, max_depth=6)
# knn = KNeighborsRegressor(20, weights = 'uniform', )

XGB = xgb.XGBRegressor(eta=0.2,
                       colsample_bytree = 0.4, 
                       learning_rate = 0.1,
                       max_depth = 9,
                       n_jobs=-1,
                       subsample=0.9,
                       n_estimators = 450)

# XGB = xgb.XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4,gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3,
#                         min_child_weight=1.5, missing=None, n_estimators=10000, nthread=-1,
#                         objective='reg:linear', reg_alpha=0.75, reg_lambda=0.45,
#                         scale_pos_weight=1, seed=42, silent=True, subsample=0.6)

### 5-b Création des features et target 

In [ ]:
feats  = df2.drop(['valeur_fonciere'], axis=1)
target = df2['valeur_fonciere']

### 5- Création de Train_test et Test_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feats, target,test_size = 0.2, random_state=15) 

### 6- Création de pipeline 


In [ ]:
# # pipe_1 = make_pipeline(column_trans, union, elastic_net)
# pipe_2 =  make_pipeline(column_trans, union, RFR)
# pipe_3 =  make_pipeline(column_trans, union, lasso)
# # pipe_4 =  make_pipeline(column_trans, union, ridge)
# pipe_5 =  make_pipeline(column_trans, union, GBR)
pipe_6 =  make_pipeline(column_trans, union, XGB)

## 7- Application de Pipeline sur les données 

In [ ]:
#  param_grid = {
#     'randomforestregressor__max_depth': np.linespace[5, 10,100]
#  }

# from sklearn.model_selection import GridSearchCV
# search = GridSearchCV(pipe, param_grid = param_grid, n_jobs=3)
# search.fit(X_train, y_train)

# print("Best parameter (CV score=%0.3f):" % search.best_score_)
# print(search.best_params_)
# search.best_score_

In [ ]:
# pipe_1.fit(X_train, y_train)
# pipe_2.fit(X_train, y_train)
# pipe_3.fit(X_train, y_train)
# pipe_4.fit(X_train, y_train)
# pipe_5.fit(X_train, y_train)
pipe_6.fit(X_train, y_train)

[11:09:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['code_departement',
                                                   'type_bien'])])),
                ('featureunion',
                 FeatureUnion(transformer_list=[('Standarisaion',
                                                 StandardScaler())])),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.4, eta=0.2, max_depth=9,
                              n_estimators=450, n_jobs=-1, subsample=0.9))])

## 8-Prédiction et évaluation du model

### 8-a Calcule de score R² 

Model Arbre de Regression test 

In [ ]:
print(pipe_6.score(X_train, y_train))
print(pipe_6.score(X_test, y_test))

0.8872274412240211
0.8375228385952218


In [ ]:
pred_train =pipe_6.predict(X_train)
pred_test = pipe_6.predict(X_test)
print('RMSE_train= ', np.sqrt(mean_squared_error(y_train, pred_train)))
print('RMSE_test= ',np.sqrt(mean_squared_error(y_test, pred_test)))

RMSE_train=  97543.882908059
RMSE_test=  116152.85450453892


In [ ]:
# score_train_XGBRegressor= pipe_6.score(X_train, y_train)
# score_test_Ela_= pipe_6.score(X_test, y_test)

# score_train_RFR = pipe_2.score(X_train, y_train)
# score_test_RFR = pipe_2.score(X_test, y_test)

# score_train_lasso = pipe_3.score(X_train, y_train)
# score_test_lasso = pipe_3.score(X_test, y_test)

# score_train_ridge = pipe_4.score(X_train, y_train)
# score_test_ridge = pipe_4.score(X_test, y_test)

# score_model = pd.DataFrame({'Elastic_Net':[score_train_Ela_Net, score_test_Ela_Net ], 'RFR': [score_train_RFR, score_test_RFR], 'lasso':[score_train_lasso, score_test_lasso],
#                             'ridge':[score_train_ridge,score_test_ridge]}, index=['train_score','test_score'])
# score_model

### 8-b Calcule de l'écart quadratique moyen (RMSE) 

In [ ]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
# print('XGBRegressor')
# pred_train =pipe_6.predict(X_train)
# pred_test = pipe_6.predict(X_test)

# MAE_train1 = mean_absolute_error(y_train, pred_train)
# MAE_test1 = mean_absolute_error(y_test, pred_test)
# MedAE_train1 = median_absolute_error(y_train, pred_train)
# MedAE_test1 = median_absolute_error(y_test, pred_test)
# RMSE_train1= np.sqrt(mean_squared_error(y_train, pred_train))
# RMSE_test1 = np.sqrt(mean_squared_error(y_test, pred_test))


# pred_train =pipe_2.predict(X_train)
# pred_test = pipe_2.predict(X_test)
# MAE_train2 = mean_absolute_error(y_train, pred_train)
# MAE_test2 = mean_absolute_error(y_test, pred_test)
# MedAE_train2 = median_absolute_error(y_train, pred_train)
# MedAE_test2 = median_absolute_error(y_test, pred_test)
# RMSE_train2 = np.sqrt(mean_squared_error(y_train, pred_train))
# RMSE_test2 = np.sqrt(mean_squared_error(y_test, pred_test))


# pred_train =pipe_3.predict(X_train)
# pred_test = pipe_3.predict(X_test)
# MAE_train3 = mean_absolute_error(y_train, pred_train)
# MAE_test3 = mean_absolute_error(y_test, pred_test)
# MedAE_train3= median_absolute_error(y_train, pred_train)
# MedAE_test3= median_absolute_error(y_test, pred_test)
# RMSE_train3= np.sqrt(mean_squared_error(y_train, pred_train))
# RMSE_test3= np.sqrt(mean_squared_error(y_test, pred_test))


# pred_train =pipe_4.predict(X_train)
# pred_test = pipe_4.predict(X_test)
# MAE_train4= mean_absolute_error(y_train, pred_train)
# MAE_test4=  mean_absolute_error(y_test, pred_test)
# MedAE_train4 = median_absolute_error(y_train, pred_train)
# MedAE_test4= median_absolute_error(y_test, pred_test)
# RMSE_train4= np.sqrt(mean_squared_error(y_train, pred_train))
# RMSE_test4= np.sqrt(mean_squared_error(y_test, pred_test))

# Metric_model = pd.DataFrame({'Elastic_Net':[MAE_train1, MAE_test1, MedAE_train1, MedAE_test1,
#                                             RMSE_train1, RMSE_test1],
#                              'RFR':[MAE_train2, MAE_test2, MedAE_train2, MedAE_test2,
#                                     RMSE_train2, RMSE_test2],
#                              'lasso':[MAE_train3, MAE_test3, MedAE_train3,MedAE_test3,
#                                     RMSE_train3, RMSE_test3],
#                              'ridge':[MAE_train4, MAE_test4, MedAE_train4,MedAE_test4,
#                                     RMSE_train4, RMSE_test4]},
                            
#                             index=['mean_absolute_error_train',
#                                    'mean_absolute_error_test',
#                                    'median_absolute_error_train',
#                                    'median_absolute_error_test', 
#                                    'sqrt(mean_squared_error_train', 
#                                    'sqrt(mean_squared_error_test'])
# Metric_model 